## 로컬 디버그 테스트

> src/train.py 파일을 이미 생성하고 나서 아래 스크립트를 진행합니다 (cloud-mode-test.ipynb 파일 참고)

라이브러리 준비

In [1]:
import os
from sagemaker.sklearn.estimator import SKLearn
import sagemaker
import boto3
from dotenv import load_dotenv
load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/dante/Library/Application Support/sagemaker/config.yaml


True

SDK 준비

In [2]:
boto3_session = boto3.Session(profile_name='awstutor')
sagemaker_session = sagemaker.Session(boto_session=boto3_session)
role = os.environ.get('SAGEMAKER_EXECUTION_ROLE_ARN')

환경변수 준비

In [3]:
# 로컬 모드 설정
os.environ['SAGEMAKER_PROGRAM'] = 'train.py'
os.environ['SAGEMAKER_SUBMIT_DIRECTORY'] = 'src'

# 환경 변수 설정 (SageMaker에서 제공하는 환경 변수 에뮬레이션)
os.environ['SM_MODEL_DIR'] = './output/model'
os.environ['SM_OUTPUT_DATA_DIR'] = './output/data'
os.environ['SM_CHANNEL_TRAIN'] = './input/data/train'
os.environ['SM_CHANNEL_TEST'] = './input/data/test'

# 필요한 디렉토리 생성
os.makedirs(os.environ['SM_MODEL_DIR'], exist_ok=True)
os.makedirs(os.environ['SM_OUTPUT_DATA_DIR'], exist_ok=True)
os.makedirs(os.environ['SM_CHANNEL_TRAIN'], exist_ok=True)
os.makedirs(os.environ['SM_CHANNEL_TEST'], exist_ok=True)

모델 및 파라미터 설정

In [5]:
# SKLearn 추정기 생성
estimator = SKLearn(
    entry_point='train.py',
    source_dir='src',
    role=role,
    # sagemaker_session=sagemaker_session,
    instance_type='local',
    framework_version='0.23-1',
    hyperparameters={
        'n-estimators': 150,
        'min-samples-split': 3
    }
)

> Docker 가 실행 가능한 환경이 되어야 합니다. Docker가 실행중인지 확인해주세요.

모델 훈련

In [10]:
# 로컬에서 학습 실행
estimator.fit({
    'train': f"file://{os.environ['SM_CHANNEL_TRAIN']}", 
    'test': f"file://{os.environ['SM_CHANNEL_TEST']}"
})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-07-29-01-50-38-921
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' found using Docker CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:sagemaker.local.image:Using the long-lived AWS credentials found in session
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-f9i9u:
    command: train
    container_name: te8camaair-algo-1-f9i9u
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Ma

time="2024-07-29T10:50:39+09:00" level=warning msg="/private/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/tmpj113iqzh/docker-compose.yaml: `version` is obsolete"
time="2024-07-29T10:50:39+09:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmpj113iqzh\".\nSet `external: true` to use an existing network"
 Container te8camaair-algo-1-f9i9u  Creating
 algo-1-f9i9u The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested 
 Container te8camaair-algo-1-f9i9u  Created
Attaching to te8camaair-algo-1-f9i9u
te8camaair-algo-1-f9i9u  | 2024-07-29 01:50:40,811 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
te8camaair-algo-1-f9i9u  | 2024-07-29 01:50:40,815 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
te8camaair-algo-1-f9i9u  | 2024-07-29 01:50:40,840 sagemaker_sklearn_container.trainin

INFO:root:copying /private/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/tmpj113iqzh/algo-1-f9i9u/output/success -> /private/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/tmpj113iqzh/artifacts/output
INFO:root:creating /private/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/tmpj113iqzh/artifacts/output/data
INFO:root:copying /private/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/tmpj113iqzh/model/scaler.joblib -> /private/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/tmpj113iqzh/artifacts/model
INFO:root:copying /private/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/tmpj113iqzh/model/model.joblib -> /private/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/tmpj113iqzh/artifacts/model


te8camaair-algo-1-f9i9u  | 2024-07-29 01:50:42,758 sagemaker-containers INFO     Reporting training SUCCESS
te8camaair-algo-1-f9i9u exited with code 0
Aborting on container exit...
 Container te8camaair-algo-1-f9i9u  Stopping
 Container te8camaair-algo-1-f9i9u  Stopped


INFO:sagemaker.local.image:===== Job Complete =====


In [15]:
# 학습된 모델을 사용한 예측
from sklearn.datasets import load_iris

# 테스트용 데이터 준비
iris = load_iris()
test_data = iris.data[:5]  # 처음 5개 샘플 사용

# 로컬 배포
predictor = estimator.deploy(instance_type='local', initial_instance_count=1)


INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-07-29-01-53-25-614
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-07-29-01-53-25-614
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using

Attaching to 0wxcb9u5u8-algo-1-g9az7
0wxcb9u5u8-algo-1-g9az7  | 2024-07-29 01:53:27,418 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
0wxcb9u5u8-algo-1-g9az7  | 2024-07-29 01:53:27,421 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
0wxcb9u5u8-algo-1-g9az7  | 2024-07-29 01:53:27,421 INFO - sagemaker-containers - nginx config: 
0wxcb9u5u8-algo-1-g9az7  | worker_processes auto;
0wxcb9u5u8-algo-1-g9az7  | daemon off;
0wxcb9u5u8-algo-1-g9az7  | pid /tmp/nginx.pid;
0wxcb9u5u8-algo-1-g9az7  | error_log  /dev/stderr;
0wxcb9u5u8-algo-1-g9az7  | 
0wxcb9u5u8-algo-1-g9az7  | worker_rlimit_nofile 4096;
0wxcb9u5u8-algo-1-g9az7  | 
0wxcb9u5u8-algo-1-g9az7  | events {
0wxcb9u5u8-algo-1-g9az7  |   worker_connections 2048;
0wxcb9u5u8-algo-1-g9az7  | }
0wxcb9u5u8-algo-1-g9az7  | 
0wxcb9u5u8-algo-1-g9az7  | http {
0wxcb9u5u8-algo-1-g9az7  |   include /etc/nginx/mime.types;
0wxcb9u5u8-algo-1-g9az7  |   default_type application/octet-stream;
0wxcb

INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 10


0wxcb9u5u8-algo-1-g9az7  | 2024-07-29 01:53:31,526 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
0wxcb9u5u8-algo-1-g9az7  | 모델 로딩 중...
0wxcb9u5u8-algo-1-g9az7  | 모델 로딩 완료
0wxcb9u5u8-algo-1-g9az7  | 192.168.65.1 - - [29/Jul/2024:01:53:32 +0000] "GET /ping HTTP/1.1" 200 0 "-" "python-urllib3/2.2.2"
!

In [16]:
predictions = predictor.predict(test_data)
print("예측 결과:", predictions)

0wxcb9u5u8-algo-1-g9az7  | 2024-07-29 01:54:41,434 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
0wxcb9u5u8-algo-1-g9az7  | 모델 로딩 중...
0wxcb9u5u8-algo-1-g9az7  | 모델 로딩 완료
0wxcb9u5u8-algo-1-g9az7  | 192.168.65.1 - - [29/Jul/2024:01:54:42 +0000] "POST /invocations HTTP/1.1" 200 168 "-" "python-urllib3/2.2.2"


예측 결과: [0 0 0 0 0]


In [17]:
# 리소스 정리
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-scikit-learn-2024-07-29-01-53-25-614
INFO:sagemaker:Deleting endpoint with name: sagemaker-scikit-learn-2024-07-29-01-53-25-614
